### Data Cleaning

- **Check for Data Stats** and Identify Columns to Operate
-
-
-
-


In [3]:
#import basic libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
unclean_basic_data = pd.read_csv("Collected_data_falcon_9.csv")

unclean_basic_data.isnull().sum()

FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass       22
Orbit              1
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad        26
Block              0
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
dtype: int64

In [10]:
# Cols to operate ---> PayloadMass , Orbit , LaunchSite

payload_mean = unclean_basic_data['PayloadMass'].mean()
unclean_basic_data['PayloadMass'].replace(to_replace=np.nan , value=payload_mean , inplace=True)

unclean_basic_data.isnull().sum()

C:\Users\chaks\AppData\Local\Temp\ipykernel_7508\4096926452.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  unclean_basic_data['PayloadMass'].replace(to_replace=np.nan , value=payload_mean , inplace=True)


FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass        0
Orbit              1
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad        26
Block              0
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
dtype: int64

In [13]:
# For orbit's 1 single missing value, we can:
#   1. Drop the row with the missing value
#   2. Replace the missing value with the mode of the column after filtering payload weigths, orbit, etc

